Link google colab : https://colab.research.google.com/drive/1PYwQzmNEy5_1W6TU00qmEgDCRiselsxj?usp=sharing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Nhom14_19120330_19120606_19120643_19120683_19120715/Preprocess_data/diemthi2020.csv')

### Tổng quan dữ liệu

In [ ]:
# Xem kích thước dữ liệu
shape = df.shape
print(f'Dữ liệu có {shape[0]} dòng và {shape[1]} biến')

Dữ liệu có 870486 dòng và 12 biến


In [ ]:
# Xem dữ liệu ở các dòng đầu tiên
df.head()

,Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd
0,0,7.00,6.50,NaN,NaN,N1,4.2,NaN,4.75,6.4,6.75,18014547
1,1,7.75,7.75,NaN,NaN,N1,2.8,NaN,3.75,7.6,6.00,18014530
2,2,6.50,NaN,NaN,NaN,NaN,NaN,NaN,4.00,4.8,4.75,18014521
3,3,8.00,9.50,NaN,NaN,N1,5.8,NaN,8.25,8.0,7.00,18014517
4,4,NaN,NaN,8.5,8.0,N1,4.0,5.0,NaN,8.2,6.50,18014523


In [ ]:
# Xem kiểu dữ liệu từng cột
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870486 entries, 0 to 870485
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        870486 non-null  int64  
 1   Dia               555072 non-null  float64
 2   GDCD              482980 non-null  float64
 3   Hoa               295536 non-null  float64
 4   Li                293287 non-null  float64
 5   Ma_mon_ngoai_ngu  772098 non-null  object 
 6   Ngoai_ngu         772098 non-null  float64
 7   Sinh              290377 non-null  float64
 8   Su                568581 non-null  float64
 9   Toan              866581 non-null  float64
 10  Van               856565 non-null  float64
 11  sbd               870486 non-null  int64  
dtypes: float64(9), int64(2), object(1)
memory usage: 79.7+ MB


+ Ta thấy dữ liệu trên có khá nhiều giá trị NaN.  
  + Tuy nhiên, đối với kì thi THPT Quốc gia, thí sinh có quyền chỉ chọn 1 trong các tổ hợp KHTN và KHXH, đồng thời cũng có các thí sinh được miễn thi môn ngoại ngữ.  
  + Nên dẫn đến dữ liệu có nhiều giá trị NaN cũng là hợp lý.
+ Kiểu dữ liệu cũng đã phù hợp trong quá trình trực quan, phân tích. Tuy nhiên, ta nên thêm 1 biến **Cum_thi** để dễ dàng phân loại thí sinh theo cụm.


### Kiểm tra và xoá các giá trị trùng nhau

In [ ]:
# Đếm các cặp giá trị trùng nhau
duplicated = df.duplicated().sum()
if duplicated > 0:
  df = df.drop_duplicates(ignore_index = True)
else: 
  print("Không có giá trị nào trùng nhau!")

Không có giá trị nào trùng nhau!


### Xử lý các giá trị NaN

##### Ở môn ngoại ngữ:
Ta thấy có số **NaN** ở **Ngoai_ngu** và **Ma_mon_ngoai_ngu** là bằng nhau, từ đó có thể suy đoán rằng các thí sinh này được **miễn thi** ngoại ngữ. Nghĩa là **không có thí sinh nào bỏ thi** môn ngoại ngữ.

In [ ]:
NaNNN = pd.isnull(df['Ngoai_ngu'])
NaNMMNN = pd.isnull(df['Ma_mon_ngoai_ngu'])

if sum(NaNNN ^ NaNMMNN) == 0:
  print("Không có thí sinh nào bị mất điểm môn ngoại ngữ")

Không có thí sinh nào bị mất điểm môn ngoại ngữ


### Thêm tên cụm thi dựa vào số báo danh

In [ ]:
# Các cụm thi và số thứ tự
map = {
    1: "Hà Nội",
    2: "TP. Hồ Chí Minh",
    3: "Hải Phòng",
    4: "Đà Nẵng",
    5: "Hà Giang",
    6: "Cao Bằng",
    7: "Lai Châu",
    8: "Lào Cai",
    9: "Tuyên Quang",
    10: "Lạng Sơn",
    11: "Bắc Kạn",
    12: "Thái Nguyên",13: "Yên Bái",14: "Sơn La",15: "Phú Thọ",16: "Vĩnh Phúc",17: "Quảng Ninh",18: "Bắc Giang",19: "Bắc Ninh",21: "Hải Dương",22: "Hưng Yên",23: "Hoà Bình",24: "Hà Nam",25: "Nam Định",26: "Thái Bình",27: "Ninh Bình",28: "Thanh Hoá",29: "Nghệ An",30: "Hà Tĩnh",31: "Quảng Bình",32: "Quảng Trị",33: "Thừa Thiên -Huế",34: "Quảng Nam",35: "Quảng Ngãi",36: "Kon Tum",37: "Bình Định",38: "Gia Lai",39: "Phú Yên",40: "Đắk Lắk",41: "Khánh Hoà",42: "Lâm Đồng",43: "Bình Phước",44: "Bình Dương",45: "Ninh Thuận",46: "Tây Ninh",47: "Bình Thuận",48: "Đồng Nai",49: "Long An",50: "Đồng Tháp",51: "An Giang",52: "Bà Rịa-Vũng Tàu",53: "Tiền Giang",54: "Kiên Giang",55: "Cần Thơ",56: "Bến Tre",57: "Vĩnh Long",58: "Trà Vinh",59: "Sóc Trăng",60: "Bạc Liêu",61: "Cà Mau",62: "Điện Biên",63: "Đăk Nông",64: "Hậu Giang",65: "Cục Nhà trường - Bộ Quốc phòng"
}

In [ ]:
# Mapping số báo danh với thứ tự cụm thi dự vào 2 số đầu của số báo danh
df["Cum_thi"] = df["sbd"].apply(lambda x: map[int(x/1_000_000)])

In [ ]:
# Xem thử một số cột đầu
df.head()

,Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd,Cum_thi
0,0,7.00,6.50,NaN,NaN,N1,4.2,NaN,4.75,6.4,6.75,18014547,Bắc Giang
1,1,7.75,7.75,NaN,NaN,N1,2.8,NaN,3.75,7.6,6.00,18014530,Bắc Giang
2,2,6.50,NaN,NaN,NaN,NaN,NaN,NaN,4.00,4.8,4.75,18014521,Bắc Giang
3,3,8.00,9.50,NaN,NaN,N1,5.8,NaN,8.25,8.0,7.00,18014517,Bắc Giang
4,4,NaN,NaN,8.5,8.0,N1,4.0,5.0,NaN,8.2,6.50,18014523,Bắc Giang


In [ ]:
# Xem số lượng thí sinh ở từng cụm
df.groupby(["Cum_thi"])["sbd"].agg('count')

Cum_thi
An Giang           15239
Bà Rịa-Vũng Tàu    11441
Bình Dương         11386
Bình Phước          9774
Bình Thuận         10893
                   ...  
Điện Biên           5608
Đăk Nông            6212
Đắk Lắk            13945
Đồng Nai           28254
Đồng Tháp          13386
Name: sbd, Length: 62, dtype: int64

#### Kết quả sau khi tiền xử lý dữ liệu.

In [ ]:
shape = df.shape
print(f'Dữ liệu có {shape[0]} dòng và {shape[1]} biến')

Dữ liệu có 870486 dòng và 13 biến


### Lưu dữ liệu đã xử lý

In [ ]:
df.head(5)

,Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd,Cum_thi
0,0,7.00,6.50,NaN,NaN,N1,4.2,NaN,4.75,6.4,6.75,18014547,Bắc Giang
1,1,7.75,7.75,NaN,NaN,N1,2.8,NaN,3.75,7.6,6.00,18014530,Bắc Giang
2,2,6.50,NaN,NaN,NaN,NaN,NaN,NaN,4.00,4.8,4.75,18014521,Bắc Giang
3,3,8.00,9.50,NaN,NaN,N1,5.8,NaN,8.25,8.0,7.00,18014517,Bắc Giang
4,4,NaN,NaN,8.5,8.0,N1,4.0,5.0,NaN,8.2,6.50,18014523,Bắc Giang


In [ ]:
df.to_csv("'/content/drive/MyDrive/data/cleaned.csv'")